In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from string import Template
from pathlib import Path
import warnings
warnings.simplefilter("ignore")
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [ ]:
path = Path('/kaggle/input/kaggle-llm-science-exam')

In [ ]:
LLM = '/kaggle/input/flan-t5/pytorch/base/2'
device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [ ]:
model = T5ForConditionalGeneration.from_pretrained(LLM).to(device)
tokenizer = T5Tokenizer.from_pretrained(LLM)

In [ ]:
test = pd.read_csv(path / 'test.csv', index_col='id')

In [ ]:
pre = \
    'Answer following questions by providing the letters A, B, C, D, & E '\
    'in order of the most likely to be correct to the to least likely to be correct.'

template = Template('$pre\n\n$prompt\n\nA) $a\nB) $b\nC) $c\nD) $d\nE) $e')

In [ ]:
def formatInput(df, idx):
    prompt = df.loc[idx, 'prompt']
    a = df.loc[idx, 'A']
    b = df.loc[idx, 'B']
    c = df.loc[idx, 'C']
    d = df.loc[idx, 'D']
    e = df.loc[idx, 'E']
    input_text = template.substitute(
        pre=pre, prompt=prompt, a=a, b=b, c=c, d=d, e=e)
    
    return input_text

In [ ]:
print(formatInput(test, 10))

In [ ]:
inputs = tokenizer(formatInput(test,10), return_tensors="pt").to(device)
outputs = model.generate(**inputs)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(answer)

In [ ]:
# Post preprocessing

In [ ]:
def post_process(predictions):
    valid = set(['A', 'B', 'C', 'D', 'E'])
    if set(predictions).isdisjoint(valid):
        final_pred = 'A B C D E'
    else:
        final_pred = []
        for prediction in predictions:
            if prediction in valid:
                final_pred += prediction
        to_add = valid - set(final_pred)
        final_pred.extend(list(to_add))
        final_pred = ' '.join(final_pred)
        # give best three answers
    return final_pred[:5]

In [ ]:
submission = pd.read_csv(path / 'sample_submission.csv', index_col='id')
for idx in test.index:
    inputs = tokenizer(formatInput(test, idx), return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    submission.loc[idx, 'prediction'] = post_process(answer)

In [ ]:
submission.to_csv('submission.csv',index=False)